In [1]:
import yt
import pandas as pd
import numpy as np
import h5py
import ytree
# yt.enable_plugins()

In [2]:
ts = yt.load("~jw254/data2/SG64-2020/DD????/output_????")
halo_ts = yt.load("~jw254/data2/SG64-2020/rockstar_halos/halos_DD????.0.bin")

In [23]:
# single_halo = yt.load('~jw254/data2/SG64-2020/rockstar_halos/halos_DD0043.0.bin')
# print(single_halo.add_particle_filter('p2'))
# ad_halo = single_halo.all_data()
# print(ad_halo['halos','particle_position'])
# print(ad_halo['halos','particle_identifier'])
# print(single_halo.field_list)


single_snapshot = yt.load('~jw254/data2/SG64-2020/DD0043/output_0043')
single_snapshot.add_particle_filter('p2')
halo_sph = single_snapshot.sphere(halo_position, halo_radius)
print(halo_sph.has_field_parameter('particle_position'))
#num_pop3 = len(halo_sph[('p2','particle_position')])

yt : [INFO     ] 2022-11-15 10:57:09,149 Parameters: current_time              = 37.656503569377
yt : [INFO     ] 2022-11-15 10:57:09,151 Parameters: domain_dimensions         = [64 64 64]
yt : [INFO     ] 2022-11-15 10:57:09,153 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2022-11-15 10:57:09,155 Parameters: domain_right_edge         = [1. 1. 1.]
yt : [INFO     ] 2022-11-15 10:57:09,157 Parameters: cosmological_simulation   = 1
yt : [INFO     ] 2022-11-15 10:57:09,159 Parameters: current_redshift          = 18.439014055127
yt : [INFO     ] 2022-11-15 10:57:09,160 Parameters: omega_lambda              = 0.6889
yt : [INFO     ] 2022-11-15 10:57:09,162 Parameters: omega_matter              = 0.3111
yt : [INFO     ] 2022-11-15 10:57:09,163 Parameters: omega_radiation           = 0
yt : [INFO     ] 2022-11-15 10:57:09,165 Parameters: hubble_constant           = 0.6766
Parsing Hierarchy : 100%|███████████████████████████████████████████████████████████████████████████

False


In [24]:
output_num=-1

num_to_skip = 25

for ds,halos in zip(ts,halo_ts):
    output_num += 1
    if output_num == 1:
        output_num = 2

    if num_to_skip > 0:
        num_to_skip -= 1
        continue
    
    # Filter out empty halo datasets
    if not ds.particles_exist or halos['num_halos'] == 0:
        print('skipping empty halo dataset')
        continue
    
    ds.add_particle_filter('p2')
    # halos.add_particle_filter('p2')
        
    ad_halos = halos.all_data()
    print(ad_halos['halos','particle_mass'])
    print(ad_halos['halos','particle_identifier'])
    
    halo_dat = {}
    #print(ds.field_list)
    for i in range(len(ad_halos['halos','particle_position'])):
        #if ad_halos['halos','particle_mass'][i] > 1e+06:
        if ad_halos['halos','particle_mass'][i] > 0:

            halo_position = ds.arr(ad_halos['halos', 'particle_position'][i].in_units('Mpccm/h').v, 'Mpccm/h')
            halo_radius = ds.arr(ad_halos['halos','virial_radius'][i].in_units('kpccm/h').v, 'kpccm/h')
            halo_id = ad_halos['halos','particle_identifier'][i]
            halo_sph = ds.sphere(halo_position, halo_radius)

            time = ds.current_time

            halo_dat[i] = {'id':halo_id, 'position':np.array(halo_position), 'radius':halo_radius, 'time':time, 'total mass':0, 'stellar mass':0, 'gas fraction':0, 'stellar mass':0, 'num pop 3 star particles':0, 'avg metallicity':0}
            
            halo_dat[i]['total mass'] = ad_halos['halos','particle_mass'][i].in_units('Msun')
            #print(halo_position)

            #print (ds.particle_types)
            #print (ds.particle_types_raw)
            #print (ds.particle_type_counts)

            # THIS LINE IS THE PROBLEM CURRENTLY
            num_pop3 = len(halo_sph[('p2','particle_position')])
            halo_dat[i]['num pop 3 star particles'] = num_pop3

            mass_gas = halo_sph.quantities.total_quantity(['cell_mass']).in_units('Msun')
            mass_tot = halo_sph.quantities.total_quantity(['cell_mass'])+halo_sph.quantities.total_quantity(['particle_mass'])
            mass_stellar = halo_sph.quantities.total_quantity(('p2','particle_mass'))

            halo_dat[i]['stellar mass'] = mass_stellar.in_units('Msun')
            halo_dat[i]['gas fraction'] = mass_gas/halo_dat[i]['total mass']

            avg_metallicity = halo_sph.quantities.weighted_average_quantity("metallicity","density")
            halo_dat[i]['avg metallicity'] = avg_metallicity

            #spin_parameter1 = halo_sph.quantities.spin_parameter()
            #spin_parameter2 = halo_sph.quantities.spin_parameter().in_units('dimensionless')
            #spin_parameter = halo_sph.quantities.weighted_average_quantity("spin_parameter", "density")
            #halo_dat[i]['spin parameter1'] = spin_parameter1
            #halo_dat[i]['spin parameter2'] = spin_parameter2
            #print(halo_sph["metallicity_fraction"])

    #pprint(halo_dat)
    
    halo_dat = pd.DataFrame.from_dict(halo_dat).transpose()
    print(halo_dat)
    print(output_num)

yt : [INFO     ] 2022-11-15 10:58:07,914 Parameters: current_time              = 0.81650998147377
yt : [INFO     ] 2022-11-15 10:58:07,918 Parameters: domain_dimensions         = [64 64 64]
yt : [INFO     ] 2022-11-15 10:58:07,921 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2022-11-15 10:58:07,923 Parameters: domain_right_edge         = [1. 1. 1.]
yt : [INFO     ] 2022-11-15 10:58:07,925 Parameters: cosmological_simulation   = 1
yt : [INFO     ] 2022-11-15 10:58:07,927 Parameters: current_redshift          = 249
yt : [INFO     ] 2022-11-15 10:58:07,928 Parameters: omega_lambda              = 0.6889
yt : [INFO     ] 2022-11-15 10:58:07,930 Parameters: omega_matter              = 0.3111
yt : [INFO     ] 2022-11-15 10:58:07,931 Parameters: omega_radiation           = 0
yt : [INFO     ] 2022-11-15 10:58:07,933 Parameters: hubble_constant           = 0.6766
yt : [INFO     ] 2022-11-15 10:58:08,043 Parameters: current_time              = 137900487911016.77 s
yt : [INF

yt : [INFO     ] 2022-11-15 10:58:09,068 Parameters: hubble_constant           = 0.6766
yt : [INFO     ] 2022-11-15 10:58:09,167 Parameters: current_time              = 878602920232696.8 s
yt : [INFO     ] 2022-11-15 10:58:09,169 Parameters: domain_dimensions         = [1 1 1]
yt : [INFO     ] 2022-11-15 10:58:09,172 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2022-11-15 10:58:09,174 Parameters: domain_right_edge         = [0.67659998 0.67659998 0.67659998]
yt : [INFO     ] 2022-11-15 10:58:09,176 Parameters: cosmological_simulation   = 1
yt : [INFO     ] 2022-11-15 10:58:09,177 Parameters: current_redshift          = 71.74271449491681
yt : [INFO     ] 2022-11-15 10:58:09,179 Parameters: omega_lambda              = 0.6888999938964844
yt : [INFO     ] 2022-11-15 10:58:09,180 Parameters: omega_matter              = 0.3111000061035156
yt : [INFO     ] 2022-11-15 10:58:09,182 Parameters: omega_radiation           = 0.0
yt : [INFO     ] 2022-11-15 10:58:09,184 Parame

yt : [INFO     ] 2022-11-15 10:58:10,246 Parameters: omega_matter              = 0.3111000061035156
yt : [INFO     ] 2022-11-15 10:58:10,248 Parameters: omega_radiation           = 0.0
yt : [INFO     ] 2022-11-15 10:58:10,249 Parameters: hubble_constant           = 0.6765999794006348
yt : [INFO     ] 2022-11-15 10:58:10,374 Parameters: current_time              = 9.5879278419673
yt : [INFO     ] 2022-11-15 10:58:10,376 Parameters: domain_dimensions         = [64 64 64]
yt : [INFO     ] 2022-11-15 10:58:10,378 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2022-11-15 10:58:10,380 Parameters: domain_right_edge         = [1. 1. 1.]
yt : [INFO     ] 2022-11-15 10:58:10,382 Parameters: cosmological_simulation   = 1
yt : [INFO     ] 2022-11-15 10:58:10,383 Parameters: current_redshift          = 47.39099946049
yt : [INFO     ] 2022-11-15 10:58:10,385 Parameters: omega_lambda              = 0.6889
yt : [INFO     ] 2022-11-15 10:58:10,386 Parameters: omega_matter          

yt : [INFO     ] 2022-11-15 10:58:11,413 Parameters: current_redshift          = 38.30764015379
yt : [INFO     ] 2022-11-15 10:58:11,414 Parameters: omega_lambda              = 0.6889
yt : [INFO     ] 2022-11-15 10:58:11,416 Parameters: omega_matter              = 0.3111
yt : [INFO     ] 2022-11-15 10:58:11,417 Parameters: omega_radiation           = 0
yt : [INFO     ] 2022-11-15 10:58:11,419 Parameters: hubble_constant           = 0.6766
yt : [INFO     ] 2022-11-15 10:58:11,524 Parameters: current_time              = 2211869029208566.0 s
yt : [INFO     ] 2022-11-15 10:58:11,526 Parameters: domain_dimensions         = [1 1 1]
yt : [INFO     ] 2022-11-15 10:58:11,528 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2022-11-15 10:58:11,530 Parameters: domain_right_edge         = [0.67659998 0.67659998 0.67659998]
yt : [INFO     ] 2022-11-15 10:58:11,532 Parameters: cosmological_simulation   = 1
yt : [INFO     ] 2022-11-15 10:58:11,534 Parameters: current_redshift      

yt : [INFO     ] 2022-11-15 10:58:12,517 Parameters: cosmological_simulation   = 1
yt : [INFO     ] 2022-11-15 10:58:12,518 Parameters: current_redshift          = 32.55459977299886
yt : [INFO     ] 2022-11-15 10:58:12,520 Parameters: omega_lambda              = 0.6888999938964844
yt : [INFO     ] 2022-11-15 10:58:12,521 Parameters: omega_matter              = 0.3111000061035156
yt : [INFO     ] 2022-11-15 10:58:12,523 Parameters: omega_radiation           = 0.0
yt : [INFO     ] 2022-11-15 10:58:12,524 Parameters: hubble_constant           = 0.6765999794006348
yt : [INFO     ] 2022-11-15 10:58:12,631 Parameters: current_time              = 17.482211362241
yt : [INFO     ] 2022-11-15 10:58:12,633 Parameters: domain_dimensions         = [64 64 64]
yt : [INFO     ] 2022-11-15 10:58:12,635 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2022-11-15 10:58:12,637 Parameters: domain_right_edge         = [1. 1. 1.]
yt : [INFO     ] 2022-11-15 10:58:12,639 Parameters: cosmolo

yt : [INFO     ] 2022-11-15 10:58:13,651 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2022-11-15 10:58:13,653 Parameters: domain_right_edge         = [1. 1. 1.]
yt : [INFO     ] 2022-11-15 10:58:13,654 Parameters: cosmological_simulation   = 1
yt : [INFO     ] 2022-11-15 10:58:13,656 Parameters: current_redshift          = 27.700630934434
yt : [INFO     ] 2022-11-15 10:58:13,657 Parameters: omega_lambda              = 0.6889
yt : [INFO     ] 2022-11-15 10:58:13,659 Parameters: omega_matter              = 0.3111
yt : [INFO     ] 2022-11-15 10:58:13,660 Parameters: omega_radiation           = 0
yt : [INFO     ] 2022-11-15 10:58:13,662 Parameters: hubble_constant           = 0.6766
yt : [INFO     ] 2022-11-15 10:58:13,782 Parameters: current_time              = 3545141862079949.0 s
yt : [INFO     ] 2022-11-15 10:58:13,785 Parameters: domain_dimensions         = [1 1 1]
yt : [INFO     ] 2022-11-15 10:58:13,787 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [

Initializing refined index: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 21.01it/s]


[14773. 26860. 64464. 61778. 47005. 10744. 33575. 12087.  2686. 14773.
 51034. 76551.] Msun/h
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11.] dimensionless


YTFieldNotFound: Could not find field ('p2', 'particle_position') in output_0026.
Did you mean:
	('all', 'particle_position')
	('io', 'particle_position')
	('nbody', 'particle_position')

In [7]:
output_num=-1

for ds,halos in zip(ts,halo_ts):
    output_num += 1
    if output_num == 1:
        output_num = 2
    
    ds.add_particle_filter('p2')
    #halos.add_particle_filter('p2')
    ad_halos = halos.all_data()
    print(ad_halos['halos','particle_mass'])
    
    halo_dat = {}
    #print(ds.field_list)
    for i in range(len(ad_halos['halos','particle_position'])):
        #if ad_halos['halos','particle_mass'][i] > 1e+06:
        if ad_halos['halos','particle_mass'][i] > 0:

            halo_position = ds.arr(ad_halos['halos', 'particle_position'][i].in_units('Mpccm/h').v, 'Mpccm/h')
            halo_radius = ds.arr(ad_halos['halos','virial_radius'][i].in_units('kpccm/h').v, 'kpccm/h')
            halo_id = ad_halos['halos','particle_identifier'][i]
            halo_sph = ds.sphere(halo_position, halo_radius)

            time = ds.current_time

            halo_dat[i] = {'id':halo_id, 'position':np.array(halo_position), 'radius':halo_radius, 'time':time, 'total mass':0, 'stellar mass':0, 'gas fraction':0, 'stellar mass':0, 'num pop 3 star particles':0, 'avg metallicity':0}
            
            halo_dat[i]['total mass'] = ad_halos['halos','particle_mass'][i].in_units('Msun')
            #print(halo_position)

            #print (ds.particle_types)
            #print (ds.particle_types_raw)
            #print (ds.particle_type_counts)

            num_pop3 = len(halo_sph[('p2','particle_position')])
            halo_dat[i]['num pop 3 star particles'] = num_pop3

            mass_gas = halo_sph.quantities.total_quantity(['cell_mass']).in_units('Msun')
            mass_tot = halo_sph.quantities.total_quantity(['cell_mass'])+halo_sph.quantities.total_quantity(['particle_mass'])
            mass_stellar = halo_sph.quantities.total_quantity(('p2','particle_mass'))

            halo_dat[i]['stellar mass'] = mass_stellar.in_units('Msun')
            halo_dat[i]['gas fraction'] = mass_gas/halo_dat[i]['total mass']

            avg_metallicity = halo_sph.quantities.weighted_average_quantity("metallicity","density")
            halo_dat[i]['avg metallicity'] = avg_metallicity

            #spin_parameter1 = halo_sph.quantities.spin_parameter()
            #spin_parameter2 = halo_sph.quantities.spin_parameter().in_units('dimensionless')
            #spin_parameter = halo_sph.quantities.weighted_average_quantity("spin_parameter", "density")
            #halo_dat[i]['spin parameter1'] = spin_parameter1
            #halo_dat[i]['spin parameter2'] = spin_parameter2
            #print(halo_sph["metallicity_fraction"])

    #pprint(halo_dat)
    
    halo_dat = pd.DataFrame.from_dict(halo_dat).transpose()
    print(halo_dat)
    print(output_num)
    
    #SAVE TO HDF5 FILE
    try:
        positions = []
        for pos in halo_dat['position']:
            positions.append(np.array(pos))
    except:
        continue
    with h5py.File("halodata/halodat_"+("DD%04i"%output_num)+".hdf5","w") as f:
        try:
            h_id = f.create_dataset("id", data=halo_dat['id'],dtype='i')

            h_pos = f.create_dataset("position", data=positions, dtype='f')
            h_pos.attrs['units'] = "Mpccm/h"

            h_t = f.create_dataset("time", data=halo_dat['time'], dtype='f')
            h_t.attrs['units'] = "code_time"

            h_r = f.create_dataset("radius", data=halo_dat['radius'], dtype='f')
            h_r.attrs['units'] = "kpccm/h"

            h_gf = f.create_dataset("gas_fraction", data=halo_dat['gas fraction'], dtype='f')
            h_gf.attrs['units'] = "dimensionless"

            h_z = f.create_dataset("avg_metallicity", data=halo_dat['avg metallicity'], dtype='f')
            h_z.attrs['units'] = "dimensionless"

            h_pop3 = f.create_dataset("num_pop3", data=halo_dat['num pop 3 star particles'], dtype='i')

            #h_spin = f.create_dataset("spin_parameter", data=halo_dat['spin parameter'], dtype='f')
            #h_spin.attrs['units'] = "dimensionless"

            h_ms = f.create_dataset("stellar_mass", data=halo_dat['stellar mass'], dtype='f')
            h_ms.attrs['units'] = "Msun"

            h_mt = f.create_dataset("total_mass", data=halo_dat['total mass'], dtype='f')
            h_mt.attrs['units'] = "Msun"
        except:
            continue

    f.close()

yt : [INFO     ] 2021-10-31 13:46:41,464 Parameters: current_time              = 0.81650998147377
yt : [INFO     ] 2021-10-31 13:46:41,471 Parameters: domain_dimensions         = [64 64 64]
yt : [INFO     ] 2021-10-31 13:46:41,473 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2021-10-31 13:46:41,476 Parameters: domain_right_edge         = [1. 1. 1.]
yt : [INFO     ] 2021-10-31 13:46:41,477 Parameters: cosmological_simulation   = 1
yt : [INFO     ] 2021-10-31 13:46:41,479 Parameters: current_redshift          = 249
yt : [INFO     ] 2021-10-31 13:46:41,480 Parameters: omega_lambda              = 0.6889
yt : [INFO     ] 2021-10-31 13:46:41,482 Parameters: omega_matter              = 0.3111
yt : [INFO     ] 2021-10-31 13:46:41,483 Parameters: omega_radiation           = 0
yt : [INFO     ] 2021-10-31 13:46:41,485 Parameters: hubble_constant           = 0.6766
yt : [INFO     ] 2021-10-31 13:46:41,508 Parameters: current_time              = 137900487911016.7 s
yt : [INFO

[] Msun/h
Empty DataFrame
Columns: []
Index: []
0


yt : [INFO     ] 2021-10-31 13:46:46,202 Allocating for 0.000e+00 particles (index particle type 'all')
yt : [INFO     ] 2021-10-31 13:46:46,206 Identified 1.000e+00 octs
yt : [INFO     ] 2021-10-31 13:46:46,609 Parameters: current_time              = 3.44792869221
yt : [INFO     ] 2021-10-31 13:46:46,611 Parameters: domain_dimensions         = [64 64 64]
yt : [INFO     ] 2021-10-31 13:46:46,614 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2021-10-31 13:46:46,616 Parameters: domain_right_edge         = [1. 1. 1.]
yt : [INFO     ] 2021-10-31 13:46:46,619 Parameters: cosmological_simulation   = 1
yt : [INFO     ] 2021-10-31 13:46:46,621 Parameters: current_redshift          = 94.692132050604
yt : [INFO     ] 2021-10-31 13:46:46,623 Parameters: omega_lambda              = 0.6889
yt : [INFO     ] 2021-10-31 13:46:46,625 Parameters: omega_matter              = 0.3111
yt : [INFO     ] 2021-10-31 13:46:46,627 Parameters: omega_radiation           = 0
yt : [INFO     ] 20

[] Msun/h
Empty DataFrame
Columns: []
Index: []
2


yt : [INFO     ] 2021-10-31 13:46:48,485 Allocating for 0.000e+00 particles (index particle type 'all')
yt : [INFO     ] 2021-10-31 13:46:48,489 Identified 1.000e+00 octs
yt : [INFO     ] 2021-10-31 13:46:48,887 Parameters: current_time              = 4.3250710024854
yt : [INFO     ] 2021-10-31 13:46:48,890 Parameters: domain_dimensions         = [64 64 64]
yt : [INFO     ] 2021-10-31 13:46:48,892 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2021-10-31 13:46:48,894 Parameters: domain_right_edge         = [1. 1. 1.]
yt : [INFO     ] 2021-10-31 13:46:48,897 Parameters: cosmological_simulation   = 1
yt : [INFO     ] 2021-10-31 13:46:48,899 Parameters: current_redshift          = 81.272160867619
yt : [INFO     ] 2021-10-31 13:46:48,901 Parameters: omega_lambda              = 0.6889
yt : [INFO     ] 2021-10-31 13:46:48,903 Parameters: omega_matter              = 0.3111
yt : [INFO     ] 2021-10-31 13:46:48,905 Parameters: omega_radiation           = 0
yt : [INFO     ] 

...

[2.9546000e+04 5.3720000e+04 2.4845600e+05 2.0145000e+04 2.1622410e+06
 2.2831000e+04 4.3110500e+05 3.0419110e+06 2.6725800e+05 9.8711000e+05
 4.0290000e+03 6.2852700e+05 3.7872800e+05 4.1099800e+05 4.4319000e+04
 5.2377000e+04 1.8399200e+05 2.6860000e+03 2.7934500e+05 7.6551000e+04
 8.5952000e+04 3.6261000e+04 4.2841900e+05 8.3266000e+04 5.6003400e+05
 3.9753000e+05 3.4918000e+04 9.5219200e+05 2.7128700e+05 3.2232000e+04
 4.0290000e+03 1.3268910e+06 1.2489900e+05 6.7821800e+05 1.3430000e+05
 7.5208000e+04 1.6116000e+04 8.0580000e+04 1.1281200e+05 6.2369260e+06
 1.2355600e+05 4.0290000e+03 4.2976000e+04 2.6860000e+04 3.6261000e+04
 2.2831000e+04 6.8493000e+04 4.0290000e+04 8.7295000e+04 6.4464000e+04
 1.5310200e+05 8.0580000e+04 3.6261000e+04 1.0609700e+05 1.5981700e+05
 1.0072500e+05 1.0878300e+05 1.7727600e+05 8.1923400e+05 3.0889000e+04
 3.8947000e+04 2.6860000e+03 2.5517000e+04 3.4918000e+04 3.3575000e+04
 1.4235800e+05 1.2892870e+06 1.0475400e+05 4.1633000e+04 1.1012600e+05
 5.506

yt : [INFO     ] 2021-10-31 16:04:27,601 Parameters: current_time              = 75.89987395114
yt : [INFO     ] 2021-10-31 16:04:27,606 Parameters: domain_dimensions         = [64 64 64]
yt : [INFO     ] 2021-10-31 16:04:27,609 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2021-10-31 16:04:27,611 Parameters: domain_right_edge         = [1. 1. 1.]
yt : [INFO     ] 2021-10-31 16:04:27,618 Parameters: cosmological_simulation   = 1
yt : [INFO     ] 2021-10-31 16:04:27,620 Parameters: current_redshift          = 11.181356517874
yt : [INFO     ] 2021-10-31 16:04:27,622 Parameters: omega_lambda              = 0.6889
yt : [INFO     ] 2021-10-31 16:04:27,623 Parameters: omega_matter              = 0.3111
yt : [INFO     ] 2021-10-31 16:04:27,625 Parameters: omega_radiation           = 0
yt : [INFO     ] 2021-10-31 16:04:27,626 Parameters: hubble_constant           = 0.6766
yt : [INFO     ] 2021-10-31 16:04:27,671 Parameters: current_time              = 1.2818747125130202e+

                      id                                           position  \
0      0.0 dimensionless  [0.2793702185153961, 0.2816590666770935, 0.253...   
1      1.0 dimensionless  [0.27762851119041443, 0.281490296125412, 0.251...   
2      2.0 dimensionless  [0.28296494483947754, 0.28554385900497437, 0.2...   
3      3.0 dimensionless  [0.29624781012535095, 0.2899778485298157, 0.27...   
4      4.0 dimensionless  [0.2800499498844147, 0.3409375250339508, 0.247...   
..                   ...                                                ...   
265  240.0 dimensionless  [0.3611270487308502, 0.3704054355621338, 0.367...   
266  241.0 dimensionless  [0.36202988028526306, 0.3778113126754761, 0.36...   
267  242.0 dimensionless  [0.363717645406723, 0.37933140993118286, 0.372...   
268  243.0 dimensionless  [0.36312681436538696, 0.37737560272216797, 0.3...   
269  244.0 dimensionless  [0.36566513776779175, 0.38440436124801636, 0.3...   

                          radius                   

Parsing Hierarchy : 100%|██████████| 777/777 [00:00<00:00, 7844.82it/s]
yt : [INFO     ] 2021-10-31 16:04:27,824 Gathering a field list (this may take a moment.)
yt : [INFO     ] 2021-10-31 16:04:32,761 Allocating for 2.640e+02 particles (index particle type 'all')
yt : [INFO     ] 2021-10-31 16:04:32,771 Identified 2.090e+02 octs


[6.7150000e+04 1.8802000e+04 6.1778000e+04 2.0145000e+04 2.2992280e+06
 4.7273800e+05 2.6725800e+05 3.0526550e+06 9.8979600e+05 4.0290000e+03
 6.2987000e+05 3.6802200e+05 2.6860000e+03 4.4319000e+04 4.0290000e+03
 2.7397300e+05 1.8399200e+05 6.1778000e+04 8.5952000e+04 4.5662000e+04
 1.6250300e+05 4.2841900e+05 6.9836000e+04 5.7212100e+05 3.9618700e+05
 6.7150300e+05 9.9382000e+04 1.0744000e+04 2.4174000e+04 2.7397300e+05
 3.8947000e+04 1.2973450e+06 1.2489900e+05 7.3462500e+05 1.3430000e+05
 7.5208000e+04 1.7459000e+04 6.4719520e+06 1.1281200e+05 8.3266000e+04
 1.2355600e+05 6.4464000e+04 1.0744000e+04 1.4773000e+05 8.4609000e+04
 6.7150000e+03 1.3832900e+05 1.2704840e+06 2.2831000e+04 3.6261000e+04
 2.8203000e+04 3.6261000e+04 3.8947000e+04 7.2522000e+04 8.7295000e+04
 7.1179000e+04 1.0744000e+05 1.0878300e+05 1.8264800e+05 8.3669300e+05
 1.1146900e+05 1.5175900e+05 3.2232000e+04 4.1633000e+04 2.1488000e+04
 1.0690330e+06 4.1901800e+05 4.0290000e+04 3.6261000e+04 2.5517000e+04
 4.163

In [55]:
ds1 = ts[-1]
halos1 = halo_ts[-1]
yt.enable_plugins()
ds.add_particle_filter('p2')
ad_halos1 = halos1.all_data()

halo_position1 = ds1.arr(ad_halos1['halos', 'particle_position'][0].in_units('Mpccm/h').v, 'Mpccm/h')
halo_radius1 = ds1.arr(ad_halos1['halos','virial_radius'][0].in_units('kpccm/h').v, 'kpccm/h')
halo_sph1 = ds1.sphere(halo_position1, halo_radius1)

#num_pop31 = len(halo_sph1[('p2','particle_position')])
#print(num_pop31)

mass_stellar1 = halo_sph1.quantities.total_quantity(('p2','particle_mass'))

yt : [INFO     ] 2021-04-19 11:40:27,262 Parameters: current_time              = 75.89987395114
yt : [INFO     ] 2021-04-19 11:40:27,264 Parameters: domain_dimensions         = [64 64 64]
yt : [INFO     ] 2021-04-19 11:40:27,265 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2021-04-19 11:40:27,267 Parameters: domain_right_edge         = [1. 1. 1.]
yt : [INFO     ] 2021-04-19 11:40:27,268 Parameters: cosmological_simulation   = 1
yt : [INFO     ] 2021-04-19 11:40:27,269 Parameters: current_redshift          = 11.181356517874
yt : [INFO     ] 2021-04-19 11:40:27,271 Parameters: omega_lambda              = 0.6889
yt : [INFO     ] 2021-04-19 11:40:27,272 Parameters: omega_matter              = 0.3111
yt : [INFO     ] 2021-04-19 11:40:27,273 Parameters: omega_radiation           = 0
yt : [INFO     ] 2021-04-19 11:40:27,275 Parameters: hubble_constant           = 0.6766
yt : [INFO     ] 2021-04-19 11:40:27,299 Parameters: current_time              = 1.2818747125130202e+

AttributeError: 'YTSphere' object has no attribute 'add_particle_filter'

In [13]:
a = ytree.load("/storage/home/hcoda1/0/jw254/data2/SG64-2020/rockstar_halos/trees/tree_0_0_0.dat")
# print(a.field_list)
def my_node(halo):
    prog = list(halo.find_root()['prog', 'uid'])
    if halo['uid'] in prog:
        color = 'red'
    else:
        color = 'black'
        
#     if halo['uid'] == 26720:
#         print(halo['Orig_halo_ID'])

    label = \
    """
    id: %d
    Orig id: %d
    mass: %.2e Msun
    """ % (halo['uid'], halo['Orig_halo_ID'], halo['mass'].to('Msun'))

    my_kwargs = {"label": label, "fontsize": 8,
                 "shape": "square", "color": color}
    return my_kwargs
p = ytree.TreePlot(a[0], dot_kwargs={'rankdir': "LR"})#, node_function=my_node)
p.save('tree_custom_nodes_1st.png')

print(halo_masses)

Loading tree roots: 100%|██████████| 7934216/7934216 [00:00<00:00, 114321832.75it/s]


RuntimeError: TreePlot requires the pydot module. You may also need to install graphviz.